In [1]:
# Дан файл с некоторыми данными. 
# Формат файла – произвольный, не совпадает с тем, что был в первом задании. 
# Необходимо считать данные и добавить их к той коллекции, куда были записаны данные в первом задании. 
# Реализовать следующие запросы:
#    вывод минимальной, средней, максимальной salary
#    вывод количества данных по представленным профессиям
#    вывод минимальной, средней, максимальной salary по городу
#    вывод минимальной, средней, максимальной salary по профессии
#    вывод минимального, среднего, максимального возраста по городу
#    вывод минимального, среднего, максимального возраста по профессии 
#    вывод максимальной заработной платы при минимальном возрасте
#    вывод минимальной заработной платы при максимальной возрасте
#    вывод минимального, среднего, максимального возраста по городу, при условии, 
#             что заработная плата больше 50 000, отсортировать вывод по любому полю.
#    вывод минимальной, средней, максимальной salary в произвольно заданных диапазонах 
#             по городу, профессии, и возрасту: 18<age<25 & 50<age<65
#    произвольный запрос с $match, $group, $sort

In [1]:
import json
from pymongo import MongoClient, collection
from bson import ObjectId

In [2]:
with open('task_2_item.json', 'r', encoding='utf-8') as file:
    json_data = json.load(file)
json_data[0:3] 

[{'job': 'Программист',
  'salary': 52296,
  'id': 5493254,
  'city': 'Касерес',
  'year': 2009,
  'age': 51},
 {'job': 'Водитель',
  'salary': 61394,
  'id': 9042910,
  'city': 'Афины',
  'year': 2006,
  'age': 46},
 {'job': 'Водитель',
  'salary': 196864,
  'id': 1303194,
  'city': 'Самора',
  'year': 2009,
  'age': 62}]

In [3]:
def read_data_from_json(file_name):
    with open(file_name, encoding='utf-8', errors='ignore') as file:
        data = json.load(file)
    return data

In [4]:
def write_data_to_mongodb(data, host, port, db_name, collection_name):
    client = MongoClient(host, port)
    db = client[db_name]
    collection = db[collection_name]
    collection.insert_many(data)

In [5]:
# вывод минимальной, средней, максимальной salary
def get_salary_stats(collection):
    aggregate_salary = collection.aggregate([
        {"$group": {
            "_id": None,
            "min_salary": {"$min": "$salary"},
            "avg_salary": {"$avg": "$salary"},
            "max_salary": {"$max": "$salary"}
        }}
    ])
    salary_stats = next(aggregate_salary)
    return salary_stats

In [6]:
# вывод количества данных по представленным профессиям
def get_data_count_by_profession(collection):
    aggregate_profession = collection.aggregate([
        {"$group": {
            "_id": "$job",
            "count": {"$sum": 1}
        }}
    ])
    profession_stats = []
    for profession in aggregate_profession:
        job = profession['_id']
        count = profession['count']
        profession_stats.append((job, count))
    return profession_stats

In [7]:
# Вывод минимальной, средней и максимальной salary по городу
def get_salary_by_city(collection):
    aggregate_salary_by_city = collection.aggregate([
        {"$group": {
            "_id": "$city",
            "min_salary": {"$min": "$salary"},
            "avg_salary": {"$avg": "$salary"},
            "max_salary": {"$max": "$salary"}
        }}
    ])

    salary_by_city = []
    for city in aggregate_salary_by_city:
        city_name = city['_id']
        min_salary = city['min_salary']
        avg_salary = city['avg_salary']
        max_salary = city['max_salary']
        salary_data = {
            "Город": city_name,
            "Минимальная зарплата": min_salary,
            "Средняя зарплата": avg_salary,
            "Максимальная зарплата": max_salary
        }
        salary_by_city.append(salary_data)
    return salary_by_city

In [8]:
# вывод минимальной, средней, максимальной salary по профессии
def get_salary_by_profession(collection):
    aggregate_salary_by_profession = collection.aggregate([
        {"$group": {
            "_id": "$job",
            "min_salary": {"$min": "$salary"},
            "avg_salary": {"$avg": "$salary"},
            "max_salary": {"$max": "$salary"}
        }}
    ])

    salary_by_profession = []
    for profession in aggregate_salary_by_profession:
        job = profession['_id']
        min_salary = profession['min_salary']
        avg_salary = profession['avg_salary']
        max_salary = profession['max_salary']
        salary_data = {
            "Профессия": job,
            "Минимальная зарплата": min_salary,
            "Средняя зарплата": avg_salary,
            "Максимальная зарплата": max_salary
        }
        salary_by_profession.append(salary_data)
    return salary_by_profession

In [9]:
# вывод минимального, среднего, максимального возраста по городу
def get_age_by_city(collection):
    aggregate_age_by_city = collection.aggregate([
        {"$group": {
            "_id": "$city",
            "min_age": {"$min": "$age"},
            "avg_age": {"$avg": "$age"},
            "max_age": {"$max": "$age"}
        }}
    ])

    age_by_city = []
    for city in aggregate_age_by_city:
        city_name = city['_id']
        min_age = city['min_age']
        avg_age = city['avg_age']
        max_age = city['max_age']
        age_data = {
            "Город": city_name,
            "Минимальный возраст": min_age,
            "Средний возраст": avg_age,
            "Максимальный возраст": max_age
        }
        age_by_city.append(age_data)
    return age_by_city

In [10]:
# вывод минимального, среднего, максимального возраста по профессии 
def get_age_by_profession(collection):
    aggregate_age_by_profession = collection.aggregate([
        {"$group": {
            "_id": "$job",
            "min_age": {"$min": "$age"},
            "avg_age": {"$avg": "$age"},
            "max_age": {"$max": "$age"}
        }}
    ])

    age_by_profession = []
    for profession in aggregate_age_by_profession:
        job_name = profession['_id']
        min_age = profession['min_age']
        avg_age = profession['avg_age']
        max_age = profession['max_age']
        age_data = {
            "Профессия": job_name,
            "Минимальный возраст": min_age,
            "Средний возраст": avg_age,
            "Максимальный возраст": max_age
        }
        age_by_profession.append(age_data)
    return age_by_profession

In [11]:
# вывод максимальной заработной платы при минимальном возрасте
def get_max_salary_by_min_age(collection):
    aggregate_salary_by_min_age = collection.aggregate([
        {"$group": {
            "_id": None,
            "min_salary": {"$min": "$salary"}
        }},
        {"$sort": {"age": 1}},
        {"$limit": 1}
    ])
    max_salary_by_min_age = next(aggregate_salary_by_min_age)['min_salary']
    return max_salary_by_min_age

In [12]:
# вывод минимальной заработной платы при максимальной возрасте
def get_min_salary_by_max_age(collection):
    aggregate_salary_by_max_age = collection.aggregate([
        {"$group": {
            "_id": None,
            "max_salary": {"$max": "$salary"}
        }},
        {"$sort": {"age": 1}},
        {"$limit": 1}
    ])
    min_salary_by_max_age = next(aggregate_salary_by_max_age)['max_salary']
    return min_salary_by_max_age

In [13]:
# вывод минимального, среднего, максимального возраста по городу, при условии, 
# что заработная плата больше 50 000, отсортировать вывод по любому полю
def get_age_stats_by_city(db, collection_name):
    pipeline = [
        {
            '$match': {
                'salary': {'$gt': 50000}
            }
        },
        {
            '$group': {
                '_id': '$city',
                'min_age': {'$min': '$age'},
                'avg_age': {'$avg': '$age'},
                'max_age': {'$max': '$age'}
            }
        },
        {
            '$sort': {'_id': 1}
        }
    ]
    result = list(db[collection_name].aggregate(pipeline))
    return result

In [14]:
# вывод минимальной, средней, максимальной salary в произвольно заданных диапазонах 
# по городу, профессии, и возрасту: 18<age<25 & 50<age<65
def get_salary_stats_by_criteria(db, collection_name):
    pipeline = [
        {
            '$match': {
                '$or': [
                    {'age': {'$gt': 18, '$lt': 25}},
                    {'age': {'$gt': 50, '$lt': 65}}
                ]
            }
        },
        {
            '$group': {
                '_id': {
                    'city': '$city',
                    'profession': '$profession',
                    'age_criteria': {
                        '$cond': [
                            {'$and': [{'$gt': ['$age', 18]}, {'$lt': ['$age', 25]}]},
                            '18-24',
                            '50-64'
                        ]
                    }
                },
                'min_salary': {'$min': '$salary'},
                'avg_salary': {'$avg': '$salary'},
                'max_salary': {'$max': '$salary'}
            }
        }
    ]
    result = list(db[collection_name].aggregate(pipeline))
    return result

In [15]:
# вывод минимального, среднего, максимального возраста по профессии, при условии, 
# что заработная плата 20000<salary<50 000, отсортировать вывод по любому полю
def get_age_stats_by_profession(db, collection_name):
    pipeline = [
        {
            '$match': {
                'salary': {'$gt': 20000, '$lt': 50000}
            }
        },
        {
            '$group': {
                '_id': '$profession',
                'min_age': {'$min': '$age'},
                'avg_age': {'$avg': '$age'},
                'max_age': {'$max': '$age'}
            }
        },
        {
            '$sort': {'_id': 1}
        }
    ]
    result = list(db[collection_name].aggregate(pipeline))
    return result

In [16]:
def default_json_encoder(obj):
    if isinstance(obj, ObjectId):
        return str(obj)
    raise TypeError(f"Object of type {obj.__class__.__name__} is not JSON serializable")

In [17]:
def write_result_to_json(result, file_path):
    with open(file_path, 'w', encoding='utf-8') as file:
        json.dump(result, file, indent=4, ensure_ascii=False, default=default_json_encoder)

In [18]:
# Параметры
file_name = 'task_2_item.json'
host = 'localhost'
port = 27017
db_name = 'mydatabase'
collection_name = 'mycollection_2'

In [19]:
data = read_data_from_json(file_name)

In [20]:
write_data_to_mongodb(data, host, port, db_name, collection_name)

In [21]:
client = MongoClient(host, port)
db = client[db_name]
collection = db[collection_name]

In [22]:
# вывод минимальной, средней, максимальной salary
salary_stats = get_salary_stats(collection)
write_result_to_json(salary_stats, 'salary_stats_2.json')

In [23]:
# вывод количества данных по представленным профессиям
profession_stats = get_data_count_by_profession(collection)
write_result_to_json(profession_stats, 'profession_stats_2.json')

In [24]:
# Вывод минимальной, средней и максимальной зарплаты по городу
salary_by_city = get_salary_by_city(collection)
write_result_to_json(salary_by_city, 'salary_by_city_2.json')

In [25]:
# вывод минимальной, средней, максимальной salary по профессии
salary_by_profession = get_salary_by_profession(collection)
write_result_to_json(salary_by_profession, 'salary_by_profession_2.json')

In [26]:
# вывод минимального, среднего, максимального возраста по городу
age_by_city = get_age_by_city(collection)
write_result_to_json(age_by_city, 'age_by_city_2.json')

In [27]:
# вывод минимального, среднего, максимального возраста по профессии 
age_by_profession = get_age_by_profession(collection)
write_result_to_json(age_by_profession, 'age_by_profession_2.json')

In [28]:
# вывод максимальной заработной платы при минимальном возрасте
max_salary_by_min_age = get_max_salary_by_min_age(collection)
write_result_to_json(max_salary_by_min_age, 'max_salary_by_min_age_2.json')

In [29]:
# вывод минимальной заработной платы при максимальной возрасте
min_salary_by_max_age = get_min_salary_by_max_age(collection)
write_result_to_json(min_salary_by_max_age, 'min_salary_by_max_age_2.json')

In [30]:
# вывод минимального, среднего, максимального возраста по городу, при условии, 
# что заработная плата больше 50 000, отсортировать вывод по любому полю
result = get_age_stats_by_city(db, collection_name)
write_result_to_json(result, 'result_1_2.json')

In [31]:
# вывод минимальной, средней, максимальной salary в произвольно заданных диапазонах 
# по городу, профессии, и возрасту: 18<age<25 & 50<age<65
result = get_salary_stats_by_criteria(db, collection_name)
write_result_to_json(result, 'result_2_2.json')

In [32]:
# вывод минимального, среднего, максимального возраста по профессии, при условии, 
# что заработная плата 20000<salary<50 000, отсортировать вывод по любому полю
result = get_age_stats_by_profession(db, collection_name)
write_result_to_json(result, 'result_3_2.json')

In [33]:
# Закрытие соединения с MongoDB
client.close()